# NLP Lab : Modèles de langue

Dans ce tp, nous allons constuire les briques principales du modèle GPT2 et entrainer un petit modèle sur des poèmes de Victor Hugo. 

Les questions sont posées dans ce notebook, puis pour executer l'entrainement, il faudra modifier le ficher `gpt_single_head.py` aussi disponible dans le reposository git.


## Données

Les données d'entrainement sont un recueil de poèmes de Victor Hugo issu du site [gutenberg.org](https://www.gutenberg.org/). Elles sont disponibles dans le répertoire `data`.

Afin de réduire la complexité du modèle, nous allons modéliser le texte au niveau caractère. Les modèles de language modélisent généralement des séquences de sous-mots en utilisant des [tokenizers](https://huggingface.co/docs/transformers/tokenizer_summary) (BPE, SentencePiece, WordPiece)

Questions :
>* En utilisant [collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter), afficher le nombre de caractères différents dans le texte et la fréquence de chaque caractère.

In [ ]:
import collections

with open('data/hugo_contemplations.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f'Number of characters in the file: {len(text)}')
##  YOUR CODE HERE

###

print (f'Number of character in counter: {sum(counter.values())}')
print (f'{len(chars)} different characters')
print (counter)


### Encodage / décodage
Afin de transformer le texte en vecteur pour le réseau de neurones, il faut encoder chaque caractère avec un entier. Les fonctions suivantes opérent l'encodage et le décodage des caractères :

In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: transform a string into a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: transform a list of integers into a string


# test that your encoder/decoder is coherent
testString = "\nDemain, dès l'aube"
assert decode(encode (testString)) ==  testString

### Découpage Train/Validation

L'objectif étant de prédire des poèmes, il ne faut pas mélanger les lignes aléatoirement. Il faut garder l'ordre des lignes dans le texte et uniquement prendre les premiers 90% pour entrainer et les 10% restants pour contrôler l'apprentissage. 

Questions :
> * Découper en `train_data` (90%) et `val_data` (10%) en utilisant du slicing sur data.

In [ ]:
import torch
# Train and validation splits
data = torch.tensor(encode(text), dtype=torch.long)
## YOUR CODE HERE
# first 90% will be train, rest val

###

### Contexte

Le modèle de langue possède comme paramètre la taille maximale du contexte à considérer pour faire la prédiction du prochain caractère. Ce contexte est appelé `block_size`. Les données d'apprentissage sont donc des séquences de caractères consécutifs, issues de l'ensemble d'entraînenement tirées aléatoirement et de longueur `block_size`.

Si le caractère de début de la séquence est `i`, la séquence de contexte est donc :
``` x = data[i:i+block_size]```
et la valeur à prédire à chaque position dans le contexte est le caractère suivant :
```y = [data[i+1:i+block_size+1]```.



In [ ]:
block_size = 8

i  = torch.randint(len(data) - block_size, (1,))
print (i)
x = train_data[i:i+block_size]
y = train_data[i+1:i+1+block_size]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print (f'context is >{decode(context.tolist())}< target is >{decode([target.tolist()])}<')

### Définition des batchs

Les batchs d'entrainement sont constitués de plusieurs séquences de caractères tirées aléatoirement dans `train_data`. Pour choisir aléatoirement une séquence à mettre dans le batch, il faut tirer aléatoirement un point de départ dans `train_data` et extraire les `block_size` caractères suivants. Lors du tirage du point de départ, faire attention à laisser suffisamment de caractères après le point de départ pour avoir une séquence de `block_size` caractères.

Questions :
> * Créer les batchs `x` en tirant `batch_size` séquences de longeur `block_size` à  partir d'un point `i` tiré aléatoirement. Empiler les exemples avec `torch.stack`.
> * Créer les batchs `y` en ajoutant le caractère suivant la séquence `x`. Empiler les exemples avec `torch.stack`.




In [ ]:
batch_size = 4
torch.manual_seed(2023)
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ## YOUR CODE HERE
    # select batch_size starting points in the data, store them in a list called starting_points

    # x is the sequence of integer starting at each straing point and of length block_size

    # y is the character after each starting position

    ### 
    # send data and target to device
    x, y = x.to(device), y.to(device)
    return x, y

### Premier modèle : un bigramme 

Le premier modèle que nous allons implémenter est un modèle bigramme. Il prédit le caractère suivant uniquement en fonction du caractère courant. Il est possible de stocker ce modèle dans une simple matrice : pour chaque caractère (en ligne), on stocke la distribution de probabilités sur l'ensemble des caractères suivants (en colonne). On peut donc le stocker dans une simple couche [`Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html).

Questions :
> * Dans le constructeur, définir une couche Embedding de taille `vocab_size` par `vocab_size`.
> * Dans forward, appliquer la couche d'embedding au batch de idx (`x`).
> * Dans forward, définir la loss comme la `cross_entropy` entre la prédiction et target (`y`).



In [ ]:
import torch.nn as nn

# use a gpu if we have one
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # we use a simple vocab_size times vocab_size tensor to store the probabilities 
        # of each token given a single token as context in nn.Embedding
        # YOUR CODE HERE

        ## 
        
    def forward(self, idx, targets=None):

        # idx and targets are both (Batch,Time) tensor of integers
        # YOUR CODE HERE

        ## 
   
        # don't compute loss if we don't have targets
        if targets is None:
            loss = None
        else:
            # change the shape of the logits and target to match what is needed for CrossEntropyLoss
            # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
            Batch, Time, Channels = logits.shape
            logits = logits.view(Batch*Time, Channels)
            targets = targets.view(Batch*Time)
            
            # negative log likelihood between prediction and target
            # YOUR CODE HERE

            ## 

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = nn.functional.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
# send the model to device
m = model.to(device)

#### Modèle avant entraînement
Le modèle n'a pas encore été entrainé, il est juste initialisé, mais on peut calculer la loss sur un batch aléatoire. Les poids étant initialisés avec une distribution normale N(0,1) sur chaque dimension, la loss attendue après l'initialisation devrait être proche de `-ln(1/vocab_size)` (l'entropie est maximale).

In [ ]:
import math
xb, yb = get_batch('train')
logits, loss = m(xb, yb)
print (logits.shape)
print (f'loss attendue {-math.log(1.0/vocab_size)}')
print (f'loss calculée {loss}')

Pour utiliser le modèle en prédiction, il faut lui fournir un premier caractère pour amorcer la séquence : c'est le prompt. Dans notre cas, on peut initialiser la génération avec le caractère de retour à la ligne pour débuter une nouvelle phrase.

Questions :
> * Créer un prompt avec un tenseur de taille (1,1) contenant l'entier correspondant au caractère `\n`.
> * Générer une séquence de caractères de taille 100 à partir de ce prompt avec les fonctions `m.generate` et `decode`.
> * Comment est la phrase générée ?

In [ ]:
print (encode(['\n']))
## YOUR CODE HERE


###

### Entrainement

Pour l'entrainement, nous utilisons un optimiseur [AdamW](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html) avec un learning rate de 1e-3. Une itération d'apprentissage consiste en 
- générer un batch
- appliquer le réseau de neurones (forward) et calculer la loss (`model(xb, yb)`)
- calculer le gradient (après avoir remis à zero le gradient cumulé) (`loss.backward()`)
- mettre à jour les paramètres (`optimizer.step()`).

In [ ]:
max_iters = 100
batch_size = 4
eval_interval = 10
learning_rate = 1e-3
eval_iters = 20

@torch.no_grad() # no gradient is computed here
def estimate_loss():
    """ Estimate the loss on eval_iters batch of train and val sets."""
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# re-create the model
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Une fois le réseau entrainé pendant 100 itérations, on peut générer une séquence de caractères.

Questions :
> * Quel est l'effet de l'entraînement ?
> * Augmenter le nombre d'itérations à 1000 puis à 10,000, noter la loss obtenue et la phrase générée. Qu'observez-vous ?

In [ ]:
idx = torch.ones((1,1), dtype=torch.long)*3
print (decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

## Single Head Attention

Nous allons maintenant implémenter le mécanisme de base de l'attention. Pour chaque couple de mots de la séquence, ce mécanisme combine Q une *query* (l'information recherchée), K une *key* (l'information obtenue) et calcule  V une *value*, un vecteur de résultat. 

![single head attention](images/single_head_attention.png)

### Masquage
Cependant, comme nous utilisons le modèle pour générer des séquences, on ne doit pas utiliser les caractères situés après le caractère courant, car ce sont justement ces caractères que l'on cherche à prédire lors de l'apprentissage : *le futur n'est pas utilisé pour prédire (le futur).* 

On va donc intégrer une matrice de masquage dans le processus. Cette matrice indique que pour le premier caractère de la séquence, on ne peux utiliser que ce caractère pour prédire (pas de contexte). Pour le second caractère, on peut utiliser le premier caractère et le second. Pour le troisième caractère, on peut utiliser le premier, le second et le troisième et ainsi de suite. Cette matrice est donc une matrice triangulaire inférieure. Cette matrice est normalisée par ligne (les lignes somment à 1).

In [ ]:
T = 8

# first version of the contraints with matrix multiplication
# create a lower triangular matrix
weights0 = torch.tril(torch.ones(T,T))
# normalize each row
weights0 = weights0 / weights0.sum(1, keepdim=True) 
print (weights0)

La couche [`softmax`](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) est une autre manière de réaliser la normalisation :

Question :
> * Vérifier qu'on obtient bien la même matrice.

In [ ]:
tril = torch.tril(torch.ones(T,T))
weights = torch.tril(torch.ones(T,T))
weights = weights.masked_fill(tril== 0, float('-inf'))
weights = nn.functional.softmax(weights, dim=-1)
print (weights)

### Implémentation

Nous pouvons maintenant implémenter la couche d'attention :

![attention_formula](images/attention_formula.png)

Questions :

> * Créer les couches key, query et value comme des couches linéaires de dimension `C` x `head_size`.
> * Appliquer les couches à `x`.
> * `weights = query x key` (transposer les deuxième et troisième dimensions de key pour pouvoir faire le produit).
> * Appliquer le facteur de normalisation.
> * Appliquer le masque triangulaire et la softmax à `weights`.
> * Appliquer value à `x`.
> * Le résultat `out` est la multiplication de `weights` par `value(x)`.


In [ ]:
head_size = 16
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
## YOUR CODE HERE
# define the Key layer  
key = 
# define the Query layer
query = 
# define the Value layer
value =  
# apply each layer to the input
k =  # (B, T, head_size)
q =  # (B, T, head_size)
v =  # (B, T, head_size)
# compute the normalize product between Q and K 
weights =  # (B, T, head_size) @ (B, 16, head_size) -> (B, T, T)
# apply the mask (lower triangular matrix)
weights = weights.masked_fill(tril== 0, float('-inf'))
# apply the softmax
weights = 
###
out  = weights @ value(x) # (B, T, head_size)

# print the result
weights[0]
out[0]

Questions :

> * Copier votre code dans `gpt_single_head.py` : la définition des couches dans le constructeur de la classe `Head` et les calculs dans la fonction `forward`.
> * Faire un entrainement.
> * Quelle loss en train et val obtenez vous ? Le texte vous parait-il meilleur ?


## Multi-head attention

La *multi-head attention* est simplement le calcul en parallèle de plusieurs *single head attention*. Chacune des single head attention est concaténée pour créer la sortie de la multi-head attention. Dans la figure issue de l'article original, le nombre de *heads* dans le *multi-head* est `h`. Afin d'opérer des combinaisons pondérées sur la sortie de chacune des single head, une couche de calcul linéaire est ajoutée.

![multi head attention](images/multi_head_attention.png)

Le code ci-dessous crée un module de multi-head attention.
Questions :
> * Dans le constructeur, créer une liste contenant `num_heads` module `Head` en utilisant la fonction [ModuleList](https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html) de pytorch.
> * Dans la fonction `forward`, appliquer chaque single head à l'input et concaténer le résultat en utilisant la fonction [cat](https://pytorch.org/docs/stable/generated/torch.cat.html) de pytorch.

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        ## YOUR CODE HERE
        ## list of num_heads modules of type Head
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        ###
        
    def forward(self, x):
        ## YOUR CODE HERE
        ## apply each head in self.heads to x and concat the results 
        out = torch.cat([h(x) for h in self.heads], dim=-1)

        return out


Questions :
> * Copier le fichier `gpt_single_head.py` en `gpt_multi_head.py`.
> * Ajouter le module MultiHeadAttention dans `gpt_multi_head.py`.
> * En tête de fichier, ajouter un paramètre  `n_head = 4`.
> * Dans le module BigramLanguageModel, remplacer le module Head par un module MultiHeadAttention avec les paramètres `num_heads = n_head` et `head_size = n_embd // n_head` pour garder le même nombre de paramètres.
> Relancer l'entrainement et noter le nombre de paramètres et les loss obtenues.

0.009893 M parameters
step 4999: train loss 2.1570, val loss 2.1802

## Ajout d'une couche de calcul FeedForward


Après les couches d'attention qui collectent l'information dans la séquence, une couche de calcul est ajoutée pour combiner toutes les informations de la séquence. Cette couche est un simple Multi-Layer-Perceptron avec une couche cachée et une non linéarité de type [RELU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html).
![multi feedfoward](images/multi_ff.png)

In [ ]:
class FeedForward(nn.Module):
    """ a simple MLP with RELU """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

Questions :
> * Ajouter le module `FeedForward` dans votre fichier `gpt_multi_head.py`.
> * Ajouter cette couche `FeedForward` après la *multi-head attention*.
> * Relancer l'entrainement et noter le nombre de paramètres et les loss obtenues.

0.010949 M parameters
step 4999: train loss 2.1290, val loss 2.1216

## Empiler les blocs

Le réseau construit jusqu'à présent n'est en fait qu'un bloc du réseau final. Il est maintenant possible d'empiler les blocs de *multi-head attention* pour créer un réseau profond. 

![multi feedfoward](images/multi_bloc.png)


Le code suivant crée un bloc : 

In [ ]:
class Block(nn.Module):
    """ A single bloc of multi-head attention """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)

    def forward(self, x):
        x = self.sa(x)
        x = self.ffwd(x)
        return x

Questions :
> * Ajouter le module `Block` dans `gpt_multi_head.py`.
> * Modifier le code de `BigramLanguageModel` pour ajouter 3 `Block(n_embd, n_head=4)` avec un container [Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) à la place de `MultiHeadAttention`et `FeedForward`.
> * Relancer l'entrainement et noter le nombre de paramètres et les loss obtenues.

0.019205 M parameters
step 4999: train loss 2.2080, val loss 2.2213

## Amélioration de l'entraînement

Si on veut continuer à augmenter la taille du réseau, il est nécessaire d'utiliser des couches permettant d'améliorer l'entraînement et ses capacités de généralisation (réduire le sur-apprentissage). Ces couches sont :
- *skip connections* ou *residual connections*
- les couches de normalisation
- le dropout.


![multi feedfoward](images/multi_skip_norm.png)


Questions :
> * Dans le module Block, ajouter une skip connection en ajoutant l'input dans chaque connexion :
```
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
```
> * Dans le module Block, ajouter 2 couches de [LayerNorm](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html) de taille `n_embd` avant la couche de `Multi-Head attention` et avant la `FeedForward`.
> * Après la série de 3 blocs, ajouter une couche de LayerNorm de taille `n_embd`.
> * Définir une variable `dropout = 0.2` en début de fichier et ajouter une couche de [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) :
>    * Après la couche RELU dans FeedForward
>    * Après la couche de MultiHead dans `MultiHeadAttention`
>    * Après la softmax dans la single head attention `Head`.
> * Relancer l'entrainement et noter le nombre de paramètres et les loss obtenues.

0.019653 M parameters

## Conclusion

Les principaux éléments de GPT2 sont en place, il faut maintenant faire passer le modèle à l'échelle et l'entraîner sur une base de données beaucoup plus grande. Pour comparaison, les paramètres de [GPT2](https://huggingface.co/transformers/v2.11.0/model_doc/gpt2.html) sont : 

* `vocab_size = 50257` : GPT2 modélise des tokens (subwords) alors que nous modélisons des caractères. Pour nous, `vocab_size = 100`.
* `n_positions = 1024` : la taille maximale du contexte. Pour nous, c'est `block_size = 8`.
* `n_embd = 768`:  la dimension des embeddings. Pour nous c'est `n_embd = 32`.
* `n_layer = 12`: le nombre de block. Pour nous c'est 3.
* `n_head = 12`: le nombre de multi-head attention. Pour nous c'est 4.

Au total, GPT2 est composé de 1,500 millions de paramètres et a été entrainé sur 8M de pages web, soit 40 Gb de texte.